# Implementación de una IA interactiva con el método de Gradiente Descendente
## Pontificia Universidad Javeriana
### Estudiante: Helver Santiago Gamboa Pacheco
### Profesor: Alexander Caicedo Dorado
En esta tarea se implementa el método de **Gradiente Descendente** para
entrenar un modelo de regresión lineal. Se utilizará un dataset de
precios de vivienda y se explorará cómo el aprendizaje depende de la
tasa de aprendizaje (η). Además, se mostrarán las gráficas de
convergencia y de comparación entre datos reales y predichos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from ipywidgets import interact, IntSlider
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from IPython.display import display

### Cargar datos
En este paso cargamos el archivo de Excel que contiene la información
de precios de vivienda y seleccionamos las variables de entrada (X)
y la variable objetivo (Y). También mostramos una tabla con los datos
en grupos de 10 filas usando un deslizador interactivo.

In [ ]:
# Carga el Excel
df = pd.read_excel("Datos.xlsx")

# Función para mostrar filas en bloques de 10
def mostrar_filas(inicio=0):
    fin = inicio + 10
    display(df.iloc[inicio:fin])

# Deslizador para mostrar los datos en grupos de 10 filas
interact(mostrar_filas, inicio=IntSlider(min=0, max=len(df)-10, step=10, value=0))

# Columnas de características y target
columnas_caracteristicas = [
    "X1 transaction date",
    "X2 house age",
    "X3 distance to the nearest MRT station",
    "X4 number of convenience stores",
    "X5 latitude",
    "X6 longitude"
]
columna_target = "Y house price of unit area"

# Matriz de características (normalizada)
scaler = StandardScaler()
A_sin_bias = scaler.fit_transform(df[columnas_caracteristicas].values)

# Agregar columna de unos (bias)
A = np.c_[np.ones(len(df)), A_sin_bias]

# Vector objetivo
y = df[columna_target].values.reshape(-1, 1)

m, n = A.shape
print("Muestras:", m, "Características (incluido bias):", n)

### Funciones del modelo
A continuación se definen las funciones necesarias:
- Hipótesis lineal
- Cálculo del costo
- Gradiente
- Gradiente Descendente

In [ ]:
def hipotesis(teta, A):
    return A @ teta

def costo(teta, A, y):
    m = len(y)
    return (1/(2*m)) * np.sum((hipotesis(teta, A) - y)**2)

def gradiente(teta, A, y):
    m = len(y)
    return (1/m) * (A.T @ (hipotesis(teta, A) - y))

def gradiente_descendente(A, y, alpha, iteraciones):
    teta = np.zeros((A.shape[1], 1))
    historial_costos = []

    for i in range(iteraciones):
        teta -= alpha * gradiente(teta, A, y)
        historial_costos.append(costo(teta, A, y))

    return teta, historial_costos

### Entrenamiento con Gradiente Descendente
Se ejecuta el algoritmo de gradiente descendente para un número de
iteraciones determinado, y se guarda la evolución del costo en cada paso.

In [ ]:
def entrenar_y_medir(alpha=0.01, iteraciones=100):
    # Entrenamos el modelo con los parámetros dados
    teta_final, historial_costos = gradiente_descendente(A, y, alpha=alpha, iteraciones=iteraciones)
    y_pred = hipotesis(teta_final, A)

    # Métricas de calidad
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)

    # Mostrar resultados
    print("===== Calidad del modelo =====")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²  : {r2:.4f}")

    # Graficamos la convergencia de la función de costo
    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.plot(historial_costos, color="blue")
    plt.xlabel("Iteraciones")
    plt.ylabel("Costo (MSE)")
    plt.title("Evolución de la función de costo")

    # Gráfico de comparación real vs predicho
    plt.subplot(1,2,2)
    plt.scatter(y, y_pred, alpha=0.6, color="green", label="Predicciones")
    plt.plot([y.min(), y.max()], [y.min(), y.max()], "r--", lw=2, label="Ideal")
    plt.xlabel("Valores reales")
    plt.ylabel("Valores predichos")
    plt.title("Comparación Real vs Predicho")
    plt.legend()

    plt.tight_layout()
    plt.show()

# Deslizador interactivo para alpha y para iteraciones
interact(entrenar_y_medir,
         alpha=(0.000001, 0.2, 0.0001),
         iteraciones=(10, 500, 10))